In [ ]:
### IMPORTS
import openmc
import numpy as np
from Meshing.Meshing import *
from Fields.Fields import *
from Kernels.Kernels import *
from Solvers.Solvers import *
from Subchannel.FluidRelation import FluidRelation
from Subchannel.Channel import Channel
from Subchannel.Channel import ChannelInterface
from Aux.CSVObjects import *
from Aux.ReactorPhysicsObjects import *

import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib.patches import Rectangle


class openmc_channel():
  def __init__(self, ch_id: int):
    self.ch_type = None
    self.power = None
    self.ch_id = ch_id
    self.x = None
    self.y = None
  def set_xy(self, x: float, y: float):
    self.x = x
    self.y = y

In [ ]:
### INTEGRATED FSRC ###
"""
Openmc tallies are integral(sigmaF*phi)dV
So we need to normalize tally data by volume eventually.
"""

### FLUID DATA ###
fluid = FluidRelation(cp=1983, mu=0.00744, k=1.44, rho_0 = 2715.13, drho_dT=-0.513)

### BC DATA ###
mdot_csv = load_csv('Data/mass_flow_startup.csv')
MDOT_MAX = 168.0
mdot_bc = MDOT_MAX / 100 * csv_interpolator(csv_df=mdot_csv, x_value=0.0, x_label='time', y_label='mdot')

### CHANNEL DATA ###
full_area = 0.000287587162368000000
half_area = 0.000143793581184000000
full_dh = 0.0158276279311347000000
half_dh = 0.0079138139655673500000

temp_tolerance = 1e-10
max_temp_iterations = 1000
nZones = 50

L0 = 0.0
L1 = 67*2.54/100

fluid=fluid
gravity = 9.81
pressure_bc = 0.0
T_bc = 900.0

fric = 'type1'
heat_source = [0.0]*nZones

### VOLUMES (cm3) ###
downcomer_vol = (np.pi * 76.862**2)*L1*100 - (np.pi * 74.299**2) * L1*100
bypass_vol = (np.pi * 71.097**2)*L1*100 - (np.pi * 70.285**2) * L1*100
up_vol = 187386.72708629695 # obtained from running volume caclulation in msreCore_r5_fission_density.py
lp_vol = 425455.22629216046 # obtained from running volume caclulation in msreCore_r5_fission_density.py
mesh_vol = (0.50885*2.0) * (L1*100/nZones) * (5.08339 - 2.0*0.50885)

### PLENUM, BYPASS, EXTERNAL LOOP & DOWNCOMER INFORMATION ###
#   (put channel geometry information in terms of m3 or m)
bypass_dh = (71.097 - 70.285)*2.0 / 100
bypass_area = bypass_vol/100.0/100.0/100.0 / (L1) # area ion m2
bypass_fric = 'annulus'
bypass_channel_rout = 71.097/100.0
bypass_channel_rin = 70.285/100.0

downcomer_dh = 1.0
downcomer_area = 0.096918919
downcomer_fric = 'none'

up_dh = 1.0 # dummy variable
up_area = 0.149106029 # length of 1.0 - 2 second residence time
up_fric = 'none'

lp_dh = 1.0  # dummy variable
lp_area = 0.149106029
lp_fric = 'none'

ex_dh = 1.0
ex_area = 0.855762105
ex_fric = 'none'

### TRACER DATA ###
tracer_names = ['c1', 'c2', 'c3', 'c4', 'c5', 'c6']
initial_value_tracers = 0.0
scheme_tracers = 'upwind'
# dnp data from openmc
decay_consts = [0.01334, 0.03273, 0.1208, 0.3029, 0.8498, 2.854]
beta = [0.000228, 0.001177, 0.001124, 0.002522, 0.001036, 0.000434]
# other tracer stuff
boundary = 'lower' # must be lower anyways
phi_tracer = 1.0
rho_tracer = 1.0


##### PRINT SOME GEOMETRY INFORMATION #####
print("bypass area m2", bypass_area)
print("bypass vol cm3", bypass_vol)
print("downcomer area m2", downcomer_area)
print("bypass vol cm3", downcomer_vol)

In [ ]:
# GET TALLY
# RUN FROM INL - PROJECTS/MOTHMAN/OPENMC_FISSION_DENSITY
sp = openmc.StatePoint('OpenMC/statepoint.450.h5')

# UP TALLY
up_tally = sp.tallies[100002].mean.reshape(1) / up_vol

# LP TALLY
lp_tally = sp.tallies[100003].mean.reshape(1) / lp_vol

# BYPASS TALLY
bypass_tally = sp.tallies[100004].mean.reshape(1) / bypass_vol

# DOWNCOMER TALLY
downcomer_tally = sp.tallies[100001].mean.reshape(1) / downcomer_vol

# TALLY DATA
print('downcomer_tally =', downcomer_tally)
print('lp_tally =', lp_tally)
print('up_tally =', up_tally)
print('bypass_tally =', bypass_tally)


# GRID TALLY
grid_tally = sp.tallies[100000]
# Get F (x*y*z, EnergyBins, 1, 1)
F = grid_tally.get_values().reshape(50,55,55)
M = grid_tally.filters[0]
xgrid = M.mesh.x_grid
ygrid = M.mesh.y_grid
zgrid = M.mesh.z_grid

# PRINT OUT INTEGRATED DATA
total_F = downcomer_tally*downcomer_vol + bypass_tally*bypass_vol + lp_tally*lp_vol + up_tally*up_vol + np.sum(F)
print('============================================')
print("total F downcomer =", downcomer_tally*downcomer_vol)
print("total F bypass =", bypass_tally*bypass_vol)
print("total F LP =", lp_tally*lp_vol)
print("total F UP =", up_tally*up_vol)
print("total F channels =", np.sum(F))
print("total F ALL =", total_F)
print('============================================')
print("fraction F downcomer =", downcomer_tally*downcomer_vol / total_F)
print("fraction F bypass =", bypass_tally*bypass_vol / total_F)
print("fraction F LP =", lp_tally*lp_vol / total_F)
print("fraction F UP =", up_tally*up_vol / total_F)
print("fraction F channels =", np.sum(F) / total_F)
print('============================================')
# NOW MAKE VALUES OF FISSION DENSITY BY DIVIDING BY VOLUME IF NOT ALREADY DONE ABOVE
# normalize so that int(F)dV == 1
downcomer_F = downcomer_tally / total_F # already normalized above
bypass_F = bypass_tally / total_F
lp_F = lp_tally / total_F
up_F = up_tally / total_F
F = F / mesh_vol / total_F # normalize to volume of each mesh voxel.
print('============================================')
print("Integrated F by vol = ", downcomer_F*downcomer_vol + bypass_F*bypass_vol + up_F * up_vol + lp_F * lp_vol + np.sum(F*mesh_vol))
print('============================================')

# integrate all to double check:




### From top to bottom - lists the number of HORIZONTAL --- channels in each row
nchannels = [3,11,15,17,19,21,23,23,25,25,25,27,27,27]
nchannels += nchannels[-1::-1]

### from top to bottom list out the yCoords
yCoords_hor = np.linspace(13.5*5.08339, -13.5*5.08339, 28) # top to bottom yCoords
xCoords_hor = np.linspace(-13*5.08339, 13*5.08339, 27) # left to right xCoords

print(nchannels)
print(sum(nchannels)*2)
len(nchannels)

### AREA RATIOS FOR CHANNELS THAT ARE HORIZONTALLY OPPOSED (FROM TOP TO BOTTOM)
channel_area_ratio_horizontal =     np.array([[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0],
                                              [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0],
                                              [0.0,0.0,0.0,0.0,0.0,0.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0],
                                              [0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0],
                                              [0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0],
                                              [0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0],
                                              [0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0],
                                              [0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0],
                                              [0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0],
                                              [0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0],
                                              [0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0],
                                              [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1],
                                              [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1],
                                              [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1],
                                              [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1],
                                              [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1],
                                              [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1],
                                              [0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0],
                                              [0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0],
                                              [0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0],
                                              [0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0],
                                              [0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0],
                                              [0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0],
                                              [0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0],
                                              [0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0],
                                              [0.0,0.0,0.0,0.0,0.0,0.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0],
                                              [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0],
                                              [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0]])
channel_area_ratio_vertical = np.transpose(channel_area_ratio_horizontal)



openmc_channel_list = []
ch_arr_list = []


### NOW GET POWER DISTRIBUTION IN EACH CHANNEL ###
power_dict = {} # Keys are channel indexes - values will be the axial power distribution.
for z in range(50):
  F_test = F[z,:,:]
  hor_arr = np.array([])
  ### HORIZONTALLY ORIENTED CHANNELS ###
  for ridx, row in enumerate(F_test[0::2, :]):
    this_nchannel = nchannels[ridx]
    middle = 27
    rightmost = 27 + (this_nchannel-1)
    leftmost = 27 - (this_nchannel-1)
    channel_vals = row[int(leftmost):int(rightmost+1):2]
    nzeros = int((27 - this_nchannel)/2)
    zeros = np.zeros(nzeros)

    channel_vals = np.append(zeros,channel_vals)
    channel_vals = np.append(channel_vals, zeros)
    print("ROW",ridx)
    print(channel_vals)
    try:
      hor_arr = np.vstack([hor_arr, channel_vals])
    except:
      hor_arr = channel_vals

  ### VERTICALLY ORIENTED CHANNELS ###
  ver_arr = np.array([])
  for cidx, col in enumerate(np.transpose(F_test)[0::2, :]):
    this_nchannel = nchannels[cidx]
    middle = 27
    rightmost = 27 + (this_nchannel-1)
    leftmost = 27 - (this_nchannel-1)
    channel_vals = col[int(leftmost):int(rightmost+1):2]
    nzeros = int((27 - this_nchannel)/2)
    zeros = np.zeros(nzeros)

    channel_vals = np.append(zeros,channel_vals)
    channel_vals = np.append(channel_vals, zeros)
    print("COL",cidx)
    print(channel_vals)
    try:
      ver_arr = np.vstack([ver_arr, channel_vals])
    except:
      ver_arr = channel_vals

  # transpose after we are done making ver_arr (since we transposed F_test while iterating.)
  ver_arr = np.transpose(ver_arr) # np.shape(ver_arr) = (27, 28) ||| hor_arr shape is 28, 27

  # Now go through and make openmc_channel objects with power appended to them.
  # also attach to openmc_channel object the channel coordinatres
  ch_idx = int(0)
  openmc_channel_list = []
  for ridx, row in enumerate(hor_arr):
    for cidx, col in enumerate(row):
      if col > 1e-15:
        this_p = col
        try:
          power_dict[ch_idx] = np.append(power_dict[ch_idx], this_p)
        except:
          power_dict[ch_idx] = np.array([this_p])
        this_ch = openmc_channel(ch_id=ch_idx)
        this_ch.ch_type = channel_area_ratio_horizontal[ridx, cidx]
        this_ch.set_xy(x=xCoords_hor[cidx], y=yCoords_hor[ridx])
        openmc_channel_list.append(this_ch)
        ch_idx += 1
  for ridx, row in enumerate(ver_arr):
    for cidx, col in enumerate(row):
      if col > 1e-15:
        this_p = col
        try:
          power_dict[ch_idx] = np.append(power_dict[ch_idx], this_p)
        except:
          power_dict[ch_idx] = np.array([this_p])
        this_ch = openmc_channel(ch_id=ch_idx)
        this_ch.ch_type = channel_area_ratio_vertical[ridx,cidx]
        this_ch.set_xy(x=yCoords_hor[cidx], y=xCoords_hor[ridx]) # flip xy since transposed compared to hor_arr
        openmc_channel_list.append(this_ch)
        ch_idx += 1

# we now have power dict with powers for each channel. channels 1-~500 are horizontally opposed channels and ~500-1152
# are vertically oriented channels.

# Now assign power shape to each of the channels
for this in openmc_channel_list:
  ch_id = this.ch_id
  this.power= power_dict[ch_id]

# Now make a channel object for this channel:
for this in openmc_channel_list:
  this_fsrc = this.power
  if this.ch_type == 0.5:
    area = half_area
    Dh = half_dh
  elif this.ch_type == 1.0:
    area = full_area
    Dh = full_dh
  else:
    raise Exception("This channel type unknown!")
  this_channel = Channel(gravity=gravity,
              Dh=Dh,
              area=area,
              temp_tolerance=temp_tolerance,
              max_temp_iterations=max_temp_iterations,
              nZones=nZones,
              L0=L0,
              L1=L1,
              fluid=fluid,
              pressure_bc=pressure_bc,
              T_bc=T_bc,
              mdot_bc=1.0,
              fric=fric,
              heat_source=heat_source)
  # set x and y coords
  this_channel.set_xy(x=this.x, y=this.y)

  # set form loss coords
  _this_r = (this.x ** 2 + this.y **2)**0.5 / 100.0
  inlet_form_loss_value = 3.81 * _this_r + 47.10
  this_channel.set_form_loss_coeffs(inlet=inlet_form_loss_value, outlet=0.5)

  # adding tracers to each channel
  for tidx, name in enumerate(tracer_names):
    this_channel.add_tracer_to_channel(name=name,
                                       initial_value=0.0,
                                       scheme=scheme_tracers,
                                       decay_const=decay_consts[tidx],
                                       boundary=boundary,
                                       phi=phi_tracer,
                                       rho=rho_tracer,
                                       source=this.power,
                                       beta=beta[tidx])
  # append channel to channel array.
  ch_arr_list.append(this_channel)


### MAKE BYPASS CHANNEL ###
bypass_channel = Channel(gravity=gravity,
              Dh=bypass_dh,
              area=bypass_area,
              temp_tolerance=temp_tolerance,
              max_temp_iterations=max_temp_iterations,
              nZones=nZones,
              L0=L0,
              L1=L1,
              fluid=fluid,
              pressure_bc=pressure_bc,
              T_bc=T_bc,
              mdot_bc=1.0,
              fric=bypass_fric,
              heat_source=heat_source)
bypass_channel.set_annulus_parameters(Rout=bypass_channel_rout, Rin=bypass_channel_rin)
bypass_channel.set_form_loss_coeffs(inlet=120.5434285, outlet=0.5)

### MAKE DOWNCOMER CHANNEL ###
downcomer_channel = Channel(gravity=gravity,
              Dh=downcomer_dh,
              area=downcomer_area,
              temp_tolerance=temp_tolerance,
              max_temp_iterations=max_temp_iterations,
              nZones=nZones,
              L0=0,
              L1=1.0,
              fluid=fluid,
              pressure_bc=0.0,
              T_bc=0.0,
              mdot_bc=0.0,
              fric=downcomer_fric,
              heat_source=heat_source)

### MAKE UPPER PLENUM CHANNEL ###
upper_plenum_channel = Channel(gravity=gravity,
              Dh=up_dh,
              area=up_area,
              temp_tolerance=temp_tolerance,
              max_temp_iterations=max_temp_iterations,
              nZones=nZones,
              L0=0,
              L1=1.0,
              fluid=fluid,
              pressure_bc=0.0,
              T_bc=1.0,
              mdot_bc=1.0,
              fric=up_fric,
              heat_source=heat_source)

### MAKE LOWER PLENUM CHANNEL ###
lower_plenum_channel = Channel(gravity=gravity,
              Dh=lp_dh,
              area=lp_area,
              temp_tolerance=temp_tolerance,
              max_temp_iterations=max_temp_iterations,
              nZones=nZones,
              L0=0,
              L1=1.0,
              fluid=fluid,
              pressure_bc=0.0,
              T_bc=1.0,
              mdot_bc=1.0,
              fric=lp_fric,
              heat_source=heat_source)

### MAKE EXTERNAL LOOP CHANNEL
ex_channel = Channel(gravity=gravity,
              Dh=ex_dh,
              area=ex_area,
              temp_tolerance=temp_tolerance,
              max_temp_iterations=max_temp_iterations,
              nZones=nZones,
              L0=0,
              L1=1.0,
              fluid=fluid,
              pressure_bc=0.0,
              T_bc=1.0,
              mdot_bc=1.0,
              fric=ex_fric,
              heat_source=heat_source)

### ADD TRACERS TO EXTRA CHANNELS ###
for tidx, name in enumerate(tracer_names):
  bypass_channel.add_tracer_to_channel(name=name,
                                       initial_value=0.0,
                                       scheme=scheme_tracers,
                                       decay_const=decay_consts[tidx],
                                       boundary=boundary,
                                       phi=phi_tracer,
                                       rho=rho_tracer,
                                       source=bypass_F,
                                       beta=beta[tidx])
  downcomer_channel.add_tracer_to_channel(name=name,
                                      initial_value=0.0,
                                      scheme=scheme_tracers,
                                      decay_const=decay_consts[tidx],
                                      boundary=boundary,
                                      phi=phi_tracer,
                                      rho=rho_tracer,
                                      source=downcomer_F,
                                      beta=beta[tidx])
  upper_plenum_channel.add_tracer_to_channel(name=name,
                                       initial_value=0.0,
                                       scheme=scheme_tracers,
                                       decay_const=decay_consts[tidx],
                                       boundary=boundary,
                                       phi=phi_tracer,
                                       rho=rho_tracer,
                                       source=up_F,
                                       beta=beta[tidx])
  lower_plenum_channel.add_tracer_to_channel(name=name,
                                       initial_value=0.0,
                                       scheme=scheme_tracers,
                                       decay_const=decay_consts[tidx],
                                       boundary=boundary,
                                       phi=phi_tracer,
                                       rho=rho_tracer,
                                       source=lp_F,
                                       beta=beta[tidx])
  ex_channel.add_tracer_to_channel(name=name,
                                       initial_value=0.0,
                                       scheme=scheme_tracers,
                                       decay_const=decay_consts[tidx],
                                       boundary=boundary,
                                       phi=phi_tracer,
                                       rho=rho_tracer,
                                       source=0.0,
                                       beta=beta[tidx])

### MAKE CHANNEL ARRAY ###
ch = ChannelArray(channels=ch_arr_list+[bypass_channel],
                  coupling_method='pressure_method',
                  flow_ratios=[],
                  fluid=fluid,
                  mdot_relaxation=1.0,
                  epsilon=1e-10)
ch.mdot_bc = MDOT_MAX

### MAKE CHANNEL CONNECTIONS
ch_to_up = ChannelInterface(ch1=ch, ch2=upper_plenum_channel)
up_to_ex = ChannelInterface(ch1=upper_plenum_channel, ch2=ex_channel)
ex_to_downcomer = ChannelInterface(ch1=ex_channel, ch2=downcomer_channel)
downcomer_to_lp = ChannelInterface(ch1=downcomer_channel, ch2=lower_plenum_channel)
lp_to_ch = ChannelInterface(ch1=lower_plenum_channel, ch2=ch)

### SOLVE THERMAL HYDRAULICS ###
ch.solve_channel_TH(_dt=1e321)
ch_to_up.update_interface_conditions(tracer_bool=False, th_bool=True)

upper_plenum_channel.solve_channel_TH(_dt=1e321)
up_to_ex.update_interface_conditions(tracer_bool=False, th_bool=True)

ex_channel.solve_channel_TH(_dt=1e321)
ex_to_downcomer.update_interface_conditions(tracer_bool=False, th_bool=True)

downcomer_channel.solve_channel_TH(_dt=1e321)
downcomer_to_lp.update_interface_conditions(tracer_bool=False, th_bool=True)

lower_plenum_channel.solve_channel_TH(_dt=1e321)
lp_to_ch.update_interface_conditions(tracer_bool=False, th_bool=True)

# End
print(ch.get_channel_residence_time())
print(upper_plenum_channel.get_channel_residence_time())
print(ex_channel.get_channel_residence_time())
print(downcomer_channel.get_channel_residence_time())
print(lower_plenum_channel.get_channel_residence_time())


In [ ]:
ch.plot_map(var='entry_loss', zNode=0, figsize=(6,6), radius=1, cmap_label='default',
            cmap_fontsize=15, cmap_name='jet',
            y_label='y (cm)', x_label='x (cm)')

In [ ]:
ch.plot_map(var='mdot', zNode=0, figsize=(6,6), radius=1, cmap_label='default',
            cmap_fontsize=15, cmap_name='jet', cmap_minmax=(0.12,0.16),
            y_label='y (cm)', x_label='x (cm)')

In [ ]:
ch.plot_map(var='int_fission_src', zNode=0, figsize=(6,6), radius=1, cmap_label='default',
            cmap_fontsize=15, cmap_name='jet', cmap_minmax='default',
            y_label='y (cm)', x_label='x (cm)')

In [ ]:
plt.figure(figsize=(2,2))
plt.pcolormesh(xgrid, ygrid, F[24,:,:])


In [ ]:
plt.figure(figsize=(2,2))
plt.pcolormesh(ver_arr)

In [ ]:
plt.figure(figsize=(2,2))
plt.pcolormesh(hor_arr)

In [ ]:
plt.figure(figsize=(3,2))
plt.plot(power_dict[200])
# plt.ylim([0,6.2e-6])

In [ ]:
plt.figure(figsize=(2,2))
plt.pcolormesh(M.mesh.volumes[:,:,0])